## Loading data

In [2]:
raw_data = readlines("octopusGrid.txt");

In [3]:
octoGrid = Array{Int}(undef,length(raw_data),length(raw_data[1]))
for i in 1:length(raw_data)
    octoGrid[i,:] = parse.(Int,split(raw_data[i],""))
end

In [4]:
octoGridTest = [ 5 4 8 3 1 4 3 2 2 3;
2 7 4 5 8 5 4 7 1 1;
5 2 6 4 5 5 6 1 7 3;
6 1 4 1 3 3 6 1 4 6;
6 3 5 7 3 8 5 4 7 8;
4 1 6 7 5 2 4 6 4 5;
2 1 7 6 8 4 1 7 2 1;
6 8 8 2 8 8 1 1 3 4;
4 8 4 6 8 4 8 5 5 4;
5 2 8 3 7 5 1 5 2 6];

## Part 1

In [5]:
#function to trigger flash in octopus flashIndx and transfer energy to surrounding octopusses
function octoFlash(grid,indx,hasFlashed)
    
    grid[indx...] +=1
    #check if octopus can flash, if yes, set hasFlashed and transfer energy
    if grid[indx...]>9 
        hasFlashed[indx...] = true
        grid[indx...] = 0
        for i in -1:1, j in -1:1
            newIndx = indx + [i,j]
            if any(newIndx .<1) || any(newIndx .> collect(size(grid)))
                continue  
            end
            if !hasFlashed[newIndx...] #automatically excludes 0,0
                octoFlash(grid,newIndx,hasFlashed)
            end
        end
    end
end

function gridIterate(init_grid,tmax)
    
    grid = copy(init_grid)
    hasFlashed = fill(false,size(grid))
    m,n = size(grid)
    nFlashes = 0
    for t in 1:tmax
        #increment all octopusses and start flash chains
        for i in 1:m, j in 1:n
            if !hasFlashed[i,j]
                octoFlash(grid,[i,j],hasFlashed)
            end
        end
        nFlashes += sum( hasFlashed) #find all flashes and update total flash counter
        fill!(hasFlashed,false) #reset flash grid
    end
    return nFlashes,grid
end

gridIterate (generic function with 1 method)

In [24]:
nflashes,octoGridNew = gridIterate(octoGridTest,100);

In [6]:
octoGridNew

10×10 Array{Int64,2}:
 0  3  9  7  6  6  6  8  6  6
 0  7  4  9  7  6  6  9  1  8
 0  0  5  3  9  7  6  9  3  3
 0  0  0  4  2  9  7  8  2  2
 0  0  0  4  2  2  9  8  9  2
 0  0  5  3  2  2  2  8  7  7
 0  5  3  2  2  2  2  9  6  6
 9  3  2  2  2  2  8  9  6  6
 7  9  2  2  2  8  6  8  6  6
 6  7  8  9  9  9  8  7  6  6

In [102]:
nflashes

1656

In [7]:
nflashes,octoGridNew = gridIterate(octoGrid,100);

In [8]:
nflashes

1647

In [9]:
octoGridNew

10×10 Array{Int64,2}:
 4  4  5  6  9  9  1  1  1  7
 4  5  7  1  1  1  1  1  1  1
 5  7  3  4  6  1  1  1  1  1
 6  2  2  2  3  4  7  1  1  1
 9  2  2  2  2  3  6  1  1  1
 8  2  2  2  3  5  1  1  1  1
 8  2  2  3  5  1  1  1  1  1
 9  3  3  5  1  1  1  1  1  1
 0  6  6  1  1  1  1  1  2  2
 0  0  2  1  1  1  1  1  2  0

## Part 2

In [7]:
function gridIterateSynchro(init_grid)
    
    grid = copy(init_grid)
    hasFlashed = fill(false,size(grid))
    m,n = size(grid)
    
    t = 0
    synchronized =false
    while !synchronized
        t+=1
        #increment all octopusses and start flash chains
        for i in 1:m, j in 1:n
            if !hasFlashed[i,j]
                octoFlash(grid,[i,j],hasFlashed)
            end
        end
        #check how many flashed, set synchronized flag if all flashed
        nflashes = sum(hasFlashed) 
        if nflashes == n*m
            synchronized = true
        end
        fill!(hasFlashed,false) #reset flash grid
    end
    return t
end

gridIterateSynchro (generic function with 2 methods)

In [8]:
tsyncTest = gridIterateSynchro(octoGridTest)

195

In [9]:
tsyncTest = gridIterateSynchro(octoGrid)

348